# Occupation

### Introduction:

Special thanks to: https://github.com/justmarkham for sharing the dataset and materials.

### Step 1. Import the necessary libraries

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,317 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,687 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-sec

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user).

### Step 3. Assign it to a variable called users.

In [4]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user', sep = '|')

users = spark.createDataFrame(df)

In [5]:
users.show(5)

+-------+---+------+----------+--------+
|user_id|age|gender|occupation|zip_code|
+-------+---+------+----------+--------+
|      1| 24|     M|technician|   85711|
|      2| 53|     F|     other|   94043|
|      3| 23|     M|    writer|   32067|
|      4| 24|     M|technician|   43537|
|      5| 33|     F|     other|   15213|
+-------+---+------+----------+--------+
only showing top 5 rows



### Step 4. Discover what is the mean age per occupation

In [10]:
from pyspark.sql.functions import round
users.groupBy('occupation').avg('age').withColumn('average_age', round('avg(age)',0)).select('occupation','average_age').show()

+-------------+-----------+
|   occupation|average_age|
+-------------+-----------+
|    librarian|       40.0|
|      retired|       63.0|
|       lawyer|       37.0|
|         none|       27.0|
|       writer|       36.0|
|   programmer|       33.0|
|    marketing|       38.0|
|        other|       35.0|
|    executive|       39.0|
|    scientist|       36.0|
|      student|       22.0|
|     salesman|       36.0|
|       artist|       31.0|
|   technician|       33.0|
|administrator|       39.0|
|     engineer|       36.0|
|   healthcare|       42.0|
|     educator|       42.0|
|entertainment|       29.0|
|    homemaker|       33.0|
+-------------+-----------+
only showing top 20 rows



### Step 5. Discover the Male ratio per occupation and sort it from the most to the least

In [29]:
denom = users.groupBy('occupation').count().withColumnRenamed('count', 'total_occupation')
num = users.filter(users.gender == 'M').groupBy('occupation').count().withColumnRenamed('count', 'male_occupation')

male_ratio = denom.join(num, on = 'occupation', how = 'inner')

male_ratio.withColumn('ratio', (male_ratio.male_occupation)/(male_ratio.total_occupation)).orderBy('ratio', ascending = False).show(n = male_ratio.count())

+-------------+----------------+---------------+-------------------+
|   occupation|total_occupation|male_occupation|              ratio|
+-------------+----------------+---------------+-------------------+
|       doctor|               7|              7|                1.0|
|     engineer|              67|             65| 0.9701492537313433|
|   technician|              27|             26| 0.9629629629629629|
|      retired|              14|             13| 0.9285714285714286|
|   programmer|              66|             60| 0.9090909090909091|
|    executive|              32|             29|            0.90625|
|    scientist|              31|             28| 0.9032258064516129|
|entertainment|              18|             16| 0.8888888888888888|
|       lawyer|              12|             10| 0.8333333333333334|
|     salesman|              12|              9|               0.75|
|     educator|              95|             69| 0.7263157894736842|
|      student|             196|  

### Step 6. For each occupation, calculate the minimum and maximum ages

In [30]:
max_age = users.groupBy('occupation').max('age')
min_age = users.groupBy('occupation').min('age')

max_age.join(min_age, 'occupation','inner').show()


+-------------+--------+--------+
|   occupation|max(age)|min(age)|
+-------------+--------+--------+
|    librarian|      69|      23|
|      retired|      73|      51|
|       lawyer|      53|      21|
|         none|      55|      11|
|       writer|      60|      18|
|   programmer|      63|      20|
|    marketing|      55|      24|
|        other|      64|      13|
|    executive|      69|      22|
|    scientist|      55|      23|
|      student|      42|       7|
|     salesman|      66|      18|
|       artist|      48|      19|
|   technician|      55|      21|
|administrator|      70|      21|
|     engineer|      70|      22|
|   healthcare|      62|      22|
|     educator|      63|      23|
|entertainment|      50|      15|
|    homemaker|      50|      20|
+-------------+--------+--------+
only showing top 20 rows



### Step 7. For each combination of occupation and gender, calculate the mean age

In [34]:
users.groupBy('occupation','gender').avg('age').orderBy(['occupation','gender']).show()

+-------------+------+------------------+
|   occupation|gender|          avg(age)|
+-------------+------+------------------+
|administrator|     F|40.638888888888886|
|administrator|     M| 37.16279069767442|
|       artist|     F|30.307692307692307|
|       artist|     M|32.333333333333336|
|       doctor|     M| 43.57142857142857|
|     educator|     F| 39.11538461538461|
|     educator|     M| 43.10144927536232|
|     engineer|     F|              29.5|
|     engineer|     M|              36.6|
|entertainment|     F|              31.0|
|entertainment|     M|              29.0|
|    executive|     F|              44.0|
|    executive|     M|38.172413793103445|
|   healthcare|     F| 39.81818181818182|
|   healthcare|     M|              45.4|
|    homemaker|     F|34.166666666666664|
|    homemaker|     M|              23.0|
|       lawyer|     F|              39.5|
|       lawyer|     M|              36.2|
|    librarian|     F|              40.0|
+-------------+------+------------

### Step 8.  For each occupation present the percentage of women and men

In [35]:
total_occ = users.groupBy('occupation').count().withColumnRenamed('count','total_occupation')
men_occ = users.filter(users.gender == "M").groupby('occupation').count().withColumnRenamed('count','men_occ')
women_occ = users.filter(users.gender == "F").groupby('occupation').count().withColumnRenamed('count','women_occ')

In [36]:
total_joined = total_occ.join(men_occ,'occupation','left').join(women_occ,'occupation','left')

In [40]:
total_joined.withColumns({'men_perc': (total_joined.men_occ/total_joined.total_occupation)*100,'women_perc':(total_joined.women_occ/total_joined.total_occupation)*100}).orderBy('occupation').show()

+-------------+----------------+-------+---------+------------------+------------------+
|   occupation|total_occupation|men_occ|women_occ|          men_perc|        women_perc|
+-------------+----------------+-------+---------+------------------+------------------+
|administrator|              79|     43|       36| 54.43037974683544| 45.56962025316456|
|       artist|              28|     15|       13| 53.57142857142857| 46.42857142857143|
|       doctor|               7|      7|     NULL|             100.0|              NULL|
|     educator|              95|     69|       26| 72.63157894736842|27.368421052631582|
|     engineer|              67|     65|        2| 97.01492537313433|2.9850746268656714|
|entertainment|              18|     16|        2| 88.88888888888889| 11.11111111111111|
|    executive|              32|     29|        3|            90.625|             9.375|
|   healthcare|              16|      5|       11|             31.25|             68.75|
|    homemaker|      